# Partie 2 : Chargement des données sur Neo4j (10 points)

#### Question : Quelles données chargez-vous dans neo4j?

Après le nettoyage, les données ont été chargées dans Neo4j pour permettre des requêtes complexes. Voici les étapes suivit :

- Création du projet dans Neo4j
- Création du dbms (base de données local)
- Identification de l’emplacement du répertoire ‘’import’’ soit dans le terminal ou le faire manuellement dans l’explorateur de fichier et y placer le fichier « Twitch_clean.csv ».

#### Question : Faites-vous des traitements/modifications lors du chargement?

Avant de charger les données on doit s’assurer de travailler dans une base de données vide.

Exécuter la commande :: 
- match(n) detach delete n

#### Chargez vos données en utilisant uniquement NEO4J

Chargement du fichier CSV :: 
- LOAD CSV WITH HEADERS FROM "file:///Twitch_clean.csv" AS row
MERGE (user:User {user_id: toInteger(row.user_id)})
MERGE (streamer:Streamer {username: row.streamer_username})
MERGE (stream:Stream {stream_id: toInteger(row.stream_id)})
SET stream.start_time = toInteger(row.start), stream.stop_time = toInteger(row.stop)
MERGE (user)-[:WATCHED]->(stream)
MERGE (stream)-[:HOSTED_BY]->(streamer);

##### Première requête

On voudrait calcule combien de fois chaque utilisateur a regardé un streamer spécifique et identifie les relations les plus "fidèles" entre utilisateurs et streamers.

- MATCH (user:User)-[:WATCHED]->(stream:Stream)-[:HOSTED_BY]->(streamer:Streamer)
RETURN user.user_id, streamer.username, COUNT(stream) AS times_watched
ORDER BY times_watched DESC
LIMIT 5;



Explication de la requête :

- MATCH (user:User)-[:WATCHED]->(stream:Stream)-[:HOSTED_BY]->(streamer:Streamer) : 
Ici, on suit les relations entre les utilisateurs, les streams qu'ils regardent et les streamers qui les hébergent.

- COUNT(stream) : On fait le décompte de combien de fois un utilisateur a regardé les streams d'un même streamer.

- RETURN user.user_id, streamer.username, COUNT(stream) AS times_watched : Il est retourné les utilisateurs, les streamers et le nombre de fois que la relation WATCHED a été réalisée.

- LIMIT 5 : Limite les résultats aux 5 relations les plus fortes.

Objectif : Cela peut être utile pour repérer des relations significatives ou des streamers ayant des communautés très engagées.

##### Deuxième requête

Cette requête identifie les streams ayant un temps de début (start_time) et de fin (stop_time) en dehors d'une plage horaire "normale" (par exemple, de 6h à 22h).

######
- MATCH (stream:Stream)-[:HOSTED_BY]->(streamer:Streamer)
WHERE stream.start_time < 360 OR stream.start_time > 1320
RETURN stream.stream_id, streamer.username, stream.start_time, stream.stop_time
ORDER BY stream.start_time
LIMIT 10;

Explication de la requête :

* stream.start_time < 360 OR stream.start_time > 1320 : Filtre les streams démarrant avant 6h du matin ou après 22h (en minutes).

* RETURN stream.stream_id, streamer.username, stream.start_time, stream.stop_time : Affiche l'ID du stream, le streamer, et les heures de début et de fin des streams hors plage.

* ORDER BY stream.start_time : Trie les résultats par ordre chronologique d'heure de début.

Objectifs : Cette analyse peut aider à repérer des streams potentiellement atypiques ou à vérifier si les streamers s'adaptent à des audiences internationales.